In [208]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

In [209]:
df = pd.read_csv("./UH_2023/UH_2023_TRAIN.txt", sep = "|")
df = df.astype({"CAMPAÑA": str, "ID_FINCA": str, "ID_ZONA": str, "ID_ESTACION": str, "VARIEDAD": str, "MODO": str, "TIPO": str, "COLOR": str, "SUPERFICIE": str})

In [210]:
import category_encoders as ce
from sklearn.preprocessing import LabelEncoder
encoder = ce.OrdinalEncoder(cols=["ID_FINCA", "ID_ZONA", "ID_ESTACION", "VARIEDAD", "COLOR", "TIPO", "MODO"])

In [236]:
df_temporal = df[df['CAMPAÑA'] != 22]

df_temporal = df_temporal.drop(axis = 1, columns = ["CAMPAÑA", "ALTITUD"])

df_encoded = encoder.fit_transform(df_temporal)

train = df_encoded[(df["PRODUCCION"] != 0) & (df_encoded["SUPERFICIE"] != "0.0")]

to_predict = df_encoded[~((df["PRODUCCION"] != 0) & (df_encoded["SUPERFICIE"] != "0.0"))]
l_encoder = LabelEncoder()
train["SUPERFICIE"] = l_encoder.fit_transform(train["SUPERFICIE"])

to_predict = to_predict.drop(axis = 1, columns = ["SUPERFICIE"])

C:\Users\xavid\AppData\Local\Temp\ipykernel_9248\2918637620.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train["SUPERFICIE"] = l_encoder.fit_transform(train["SUPERFICIE"])


In [284]:
df_supcat = train.groupby(by='SUPERFICIE').count().reset_index()
segur_train = df_supcat[df_supcat["TIPO"] == 1]["SUPERFICIE"]
train_nou = train[train['SUPERFICIE'].isin(list(segur_train))]

In [285]:
train_a_splitejar = train.drop(index=train_nou.index)

In [286]:
X_train_a_splitejar = train_a_splitejar.drop(axis = 1, columns = ["SUPERFICIE"])
y_train_a_splitejar = train_a_splitejar["SUPERFICIE"]

In [287]:
X_train_nou = train_nou.drop(axis = 1, columns = ["SUPERFICIE"])
y_train_nou = train_nou["SUPERFICIE"]

In [288]:
X_train_splitted, X_test, y_train_splitted, y_test = train_test_split(X_train_a_splitejar, y_train_a_splitejar, test_size=0.35, random_state=3, stratify = train_a_splitejar["SUPERFICIE"])

In [289]:
X_train = pd.concat([X_train_splitted, X_train_nou])
y_train = pd.concat([y_train_splitted, y_train_nou])

In [290]:
len(X_test) + len(X_train) == len(train)

True

In [55]:
import xgboost as xgb
params_model = {'colsample_bytree': 0.4, 'gamma': 0.0, 'learning_rate': 0.15, 'max_depth': 100, 'min_child_weight': 1, 'n_estimators': 2000}
model = xgb.XGBClassifier(**params_model)
model.fit(X_train,y_train)

/opt/anaconda3/lib/python3.8/site-packages/xgboost/sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[00:23:57] WARNING: /opt/concourse/worker/volumes/live/7a2b9f41-3287-451b-6691-43e9a6c0910f/volume/xgboost-split_1619728204606/work/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


KeyboardInterrupt: 

In [29]:
print(model.score(X_test, y_test))
print(model.score(X_train, y_train))

0.7248182762201454
0.7334905660377359


In [30]:
y_test_dec_gt = l_encoder.inverse_transform(y_test)
y_predicted = l_encoder.inverse_transform(model.predict(X_test))

In [31]:
y_test_dec_gt = y_test_dec_gt.astype(np.float)
y_predicted = y_predicted.astype(np.float)

C:\Users\adars\AppData\Local\Temp\ipykernel_11060\2686677544.py:1: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  y_test_dec_gt = y_test_dec_gt.astype(np.float)
C:\Users\adars\AppData\Local\Temp\ipykernel_11060\2686677544.py:2: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  y_predicted = y_predicted.astype(np.float)


In [32]:
from sklearn.metrics import r2_score
r2_score(y_test_dec_gt, y_predicted)

0.43315455142273485

In [202]:
len(y_train)

2120

In [204]:
model.predict_proba(X_test)

array([[6.8190414e-04, 3.8513844e-04, 1.7557872e-04, ..., 5.1132516e-05,
        1.1260376e-04, 5.8873522e-04],
       [1.4687682e-04, 4.1054318e-05, 9.5589610e-05, ..., 1.7975106e-04,
        3.6124216e-05, 8.1145583e-05],
       [3.8297463e-04, 1.6848360e-04, 1.4259036e-04, ..., 2.7710092e-04,
        1.7916671e-03, 4.3455581e-04],
       ...,
       [1.2515013e-03, 3.2755014e-04, 1.6364275e-04, ..., 1.5561598e-04,
        5.9414026e-04, 2.9936555e-04],
       [2.1049769e-04, 5.8837293e-05, 2.6087352e-04, ..., 2.5761157e-04,
        5.1771684e-05, 1.1629438e-04],
       [1.4517110e-04, 1.2535493e-04, 3.9822314e-04, ..., 2.7865608e-04,
        1.0341343e-04, 4.9287215e-04]], dtype=float32)

In [43]:
X_test = X_test.reset_index()

In [51]:
X_test

,ID_FINCA,ID_ZONA,ID_ESTACION,VARIEDAD,MODO,TIPO,COLOR
8200,580,68,8,4,2,1,1
9223,541,65,14,9,2,1,2
7875,284,43,11,5,2,1,1
8166,529,65,14,7,2,1,2
8304,703,73,14,8,1,1,1
...,...,...,...,...,...,...,...
7691,1064,29,9,13,1,1,2
9312,638,72,14,2,1,1,1
8071,424,57,10,2,1,1,1
7333,803,74,13,9,2,1,2


In [78]:
llista_idx = np.where(np.array(y_test) == np.array(model.predict(X_test)))[0]

In [59]:
len(np.array(model.predict(X_test)))

963

In [74]:
y_train.nunique()

1211

In [75]:
X_test.iloc[0, :]

ID_FINCA       580
ID_ZONA         68
ID_ESTACION      8
VARIEDAD         4
MODO             2
TIPO             1
COLOR            1
Name: 8200, dtype: int32

In [89]:
model.predict_proba(X_test.iloc[llista_idx, :])[0]

array([6.8740122e-04, 3.8630271e-04, 1.7607113e-04, ..., 5.1061033e-05,
       1.1203519e-04, 5.9812365e-04], dtype=float32)

In [94]:
llista_probs = model.predict_proba(X_test.iloc[llista_idx, :])

In [98]:
np.max(llista_probs[1])

0.7917654

In [103]:
prob_max = np.array([np.max(llista_probs[i]) for i in range(len(llista_idx))])

In [106]:
len(prob_max)

698

In [117]:
dict_prob = {"Menor a 0.5": (prob_max < 0.5).sum()/len(prob_max), "Major a 0.5": (prob_max > 0.5).sum()/len(prob_max),
             "Major a 0.6": (prob_max > 0.6).sum()/len(prob_max), "Major a 0.65": (prob_max > 0.65).sum()/len(prob_max),
            "Major a 0.75": (prob_max > 0.75).sum()/len(prob_max), "Major a 0.8": (prob_max > 0.8).sum()/len(prob_max),
            "Major a 0.9": (prob_max > 0.9).sum()/len(prob_max), "Major a 0.95": (prob_max > 0.95).sum()/len(prob_max)}
dict_counts = {"Menor a 0.5": (prob_max < 0.5).sum(), "Major a 0.5": (prob_max > 0.5).sum(),
               "Major a 0.6": (prob_max > 0.6).sum(), "Major a 0.65": (prob_max > 0.65).sum(),
            "Major a 0.75": (prob_max > 0.75).sum(), "Major a 0.8": (prob_max > 0.8).sum(),
            "Major a 0.9": (prob_max > 0.9).sum(), "Major a 0.95": (prob_max > 0.95).sum()}

In [118]:
df_prob = pd.DataFrame(dict_prob, index=[0])
df_counts = pd.DataFrame(dict_counts, index=[0])

In [119]:
df_prob

,Menor a 0.5,Major a 0.5,Major a 0.6,Major a 0.65,Major a 0.75,Major a 0.8,Major a 0.9,Major a 0.95
0,0.266476,0.733524,0.232092,0.19341,0.103152,0.055874,0.018625,0.0


In [120]:
df_counts

,Menor a 0.5,Major a 0.5,Major a 0.6,Major a 0.65,Major a 0.75,Major a 0.8,Major a 0.9,Major a 0.95
0,186,512,162,135,72,39,13,0


In [130]:
(prob_max > 0.4).sum()

639

In [129]:
len(y_test)

963

In [125]:
((0.45 <= prob_max) & (prob_max <= 0.65)).sum()

456

In [126]:
#params_model2 = {'colsample_bytree': 0.4, 'gamma': 0.0, 'learning_rate': 0.1, 'max_depth': 100, 'min_child_weight': 1, 'n_estimators': 600}
param2 = {}
param2['booster'] = 'gbtree'
param2["eval_metric"] = "error"
param2['eta'] = 0.3
param2['gamma'] = 0
param2['max_depth'] = 6
param2['min_child_weight']=1
param2['max_delta_step'] = 0
param2['subsample']= 1
param2['colsample_bytree']=1
param2['seed'] = 0
param2['base_score'] = 0.5
param2["colsample_bytree"] = 0.4
param2["learning_rate"] = 0.1
param2["n_estimators"] = 2000

model2 = xgb.XGBClassifier(**param2)
model2.fit(X_train,y_train)

[23:26:26] WARNING: C:/buildkite-agent/builds/buildkite-windows-cpu-autoscaling-group-i-0fc7796c793e6356f-1/xgboost/xgboost-ci-windows/src/learner.cc:767: 
Parameters: { "silent" } are not used.



XGBClassifier(base_score=0.5, booster='gbtree', callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=0.4, early_stopping_rounds=None,
              enable_categorical=False, eta=0.3, eval_metric='error',
              feature_types=None, gamma=0, gpu_id=None, grow_policy=None,
              importance_type=None, interaction_constraints=None,
              learning_rate=0.1, max_bin=None, max_cat_threshold=None,
              max_cat_to_onehot=None, max_delta_step=0, max_depth=6,
              max_leaves=None, min_child_weight=1, missing=nan,
              monotone_constraints=None, n_estimators=2000, n_jobs=None,
              num_parallel_tree=None, objective='multi:softprob', ...)

In [127]:
print(model2.score(X_test, y_test))
print(model2.score(X_train, y_train))

0.726895119418484
0.7344339622641509


In [131]:
llista_probs = model.predict_proba(X_test)

In [132]:
llista_probs[0]

array([6.8740122e-04, 3.8630271e-04, 1.7607113e-04, ..., 5.1061033e-05,
       1.1203519e-04, 5.9812365e-04], dtype=float32)

In [156]:
thr_04 = [idx for idx,i in enumerate(llista_probs) if np.max(i) > 0.5]

In [157]:
X_test_subset = X_test.iloc[thr_04, :]

In [158]:
(np.array(y_test.iloc[thr_04]) == np.array(model.predict(X_test_subset))*1).sum()

512

In [73]:
len(y_test.iloc[thr_04])

NameError: name 'thr_04' is not defined

### Predir + assignar prediccions

In [72]:
to_predict

,ID_FINCA,ID_ZONA,ID_ESTACION,VARIEDAD,MODO,TIPO,COLOR
0,1,1,1,1,1,1,1
1,2,1,1,1,1,1,1
2,3,2,1,2,1,1,1
3,4,2,1,2,1,1,1
4,5,3,2,3,2,1,1
...,...,...,...,...,...,...,...
8598,35,8,2,5,1,1,1
8601,36,8,2,5,1,1,1
8606,39,8,2,5,1,1,1
8608,40,8,2,5,1,1,1


#### Classifier

In [ ]:
#params_model = grid_result.best_params_
param2 = {}
param2['booster'] = 'gbtree'
param2["eval_metric"] = "error"
param2['eta'] = 0.3
param2['gamma'] = 0
param2['max_depth'] = 6
param2['min_child_weight']=1
param2['max_delta_step'] = 0
param2['subsample']= 1
param2['colsample_bytree']=1
param2['seed'] = 0
param2['base_score'] = 0.5
param2["colsample_bytree"] = 0.4
param2["learning_rate"] = 0.1
param2["n_estimators"] = 2000

model2 = xgb.XGBClassifier(**param2)
model2.fit(train.drop(columns=['SUPERFICIE']), train['SUPERFICIE'])

In [ ]:
pred_class = model2.predict(to_predict)

In [ ]:
pred_proba = model2.predict_proba(to_predict)

In [ ]:
nova_col = pd.DataFrame(((pred_proba >= 0.5)*1).sum(axis=-1))
nova_col = nova_col.set_index(to_predict.index)
to_predict['SUPERFICIE'] = nova_col
to_predict['SUPERFICIE'] = pred_class*to_predict['SUPERFICIE']

In [ ]:
to_predict2 = to_predict[to_predict["SUPERFICIE"] != 0]
nou_dataset = pd.concat([train, to_predict])

In [ ]:
params_model = {'colsample_bytree': 0.4, 'gamma': 0.0, 'learning_rate': 0.15, 'max_depth': 60, 'min_child_weight': 1, 'n_estimators': 1000}
model = xgb.XGBRegressor(**params_model)
model.fit(train.drop(columns=['SUPERFICIE']), train['SUPERFICIE'])

In [ ]:
pred_reg = model.predict(to_predict)

In [254]:

nova_col2 = (to_predict['SUPERFICIE'] == 0)*1*pred_reg
to_predict['SUPERFICIE'] += nova_col2
to_predict

,ID_FINCA,ID_ZONA,ID_ESTACION,VARIEDAD,MODO,TIPO,COLOR,PRODUCCION,SUPERFICIE
0,1,1,1,1,1,1,1,22215.0,1035.799927
1,2,1,1,1,1,1,1,22215.0,1035.799927
2,3,2,1,2,1,1,1,20978.0,1186.094849
3,4,2,1,2,1,1,1,40722.0,1161.274780
4,5,3,2,3,2,1,1,14126.0,995.496887
...,...,...,...,...,...,...,...,...,...
8598,35,8,2,5,1,1,1,NaN,381.403931
8601,36,8,2,5,1,1,1,NaN,652.540344
8606,39,8,2,5,1,1,1,NaN,615.386597
8608,40,8,2,5,1,1,1,NaN,479.312744


In [260]:
sum((to_predict['SUPERFICIE'] < 0)*1)

0

In [263]:
nou_dataset = pd.concat([train, to_predict])

In [283]:
nou_dataset.tail(50)

,ID_FINCA,ID_ZONA,ID_ESTACION,VARIEDAD,MODO,TIPO,COLOR,SUPERFICIE,PRODUCCION
6468,907,103,16,2,1,1,1,851.000000,6388.395
6469,908,103,16,2,1,1,1,1021.736816,17141.605
6470,908,103,16,5,1,1,1,938.969116,13180.000
6471,909,104,18,8,1,1,1,1005.000000,22947.500
6472,911,104,18,2,1,1,1,1051.000000,30840.000
6473,911,104,18,5,1,1,1,994.429077,24090.000
6474,912,104,18,6,1,1,1,1118.586060,35920.000
6475,912,104,18,8,1,1,1,548.197754,11302.500
6479,916,7,2,5,1,1,1,754.286987,2347.488
6486,14,7,2,5,1,1,1,773.365723,4346.706


In [282]:
df[df['ID_FINCA']=='40']

,CAMPAÑA,ID_FINCA,ID_ZONA,ID_ESTACION,ALTITUD,VARIEDAD,MODO,TIPO,COLOR,SUPERFICIE,PRODUCCION
